In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, callbacks


In [ ]:

# 1. Загрузка и подготовка данных
X = np.load("images.npy").astype("float32") / 255.0
y = np.load("labels.npy").astype("int32")

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

print("Форма X:", X.shape)
print("Форма y:", y.shape) 
print("Уникальные значения y:", np.unique(y))
print("Диапазон значений X:", X.min(), X.max())

print("Размер train:", X_train.shape, y_train.shape)
print("Размер val:", X_val.shape, y_val.shape)
print("Распределение классов train:", np.bincount(y_train))
print("Распределение классов val:", np.bincount(y_val))


Форма X: (20000, 48, 48, 3)
Форма y: (20000,)
Уникальные значения y: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]
Диапазон значений X: 0.0 1.0
Размер train: (17000, 48, 48, 3) (17000,)
Размер val: (3000, 48, 48, 3) (3000,)
Распределение классов train: [636 746 604 748 649 575 694 768 374 422 675 435 826 783 740 645 736 790
 628 489 649 628 861 694 580 625]
Распределение классов val: [112 132 106 132 115 102 122 136  66  75 119  77 146 138 131 114 130 139
 111  86 114 111 152 122 102 110]


In [ ]:
# 2. Аугментация
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=2,
    width_shift_range=0.02,
    height_shift_range=0.02,
    shear_range=0.02,
    zoom_range=0.02,
    horizontal_flip=False, # Буквы не переворачиваем
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

train_gen = train_datagen.flow(X_train, y_train, batch_size=64, shuffle=True)
val_gen = val_datagen.flow(X_val, y_val, batch_size=64, shuffle=False)

# tests
sample_batch_x, sample_batch_y = next(train_gen)
print("Форма батча X:", sample_batch_x.shape)
print("Форма батча y:", sample_batch_y.shape)
print("Диапазон значений в батче:", sample_batch_x.min(), sample_batch_x.max())
print("Уникальные метки в батче:", np.unique(sample_batch_y))

print("Шагов в train_gen:", len(train_gen))
print("Шагов в val_gen:", len(val_gen))

Форма батча X: (64, 48, 48, 3)
Форма батча y: (64,)
Диапазон значений в батче: 0.0 1.0
Уникальные метки в батче: [ 1  2  3  5  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24 25]
Шагов в train_gen: 266
Шагов в val_gen: 47


In [ ]:
# 3. Модель
def make_model(input_shape=(48, 48, 3), n_classes=26):
    """
    Создает сверточную нейронную сеть для классификации букв алфавита
    
    Args:
        input_shape: размер входного изображения (высота, ширина, каналы)
        n_classes: количество классов для классификации (26 букв A-Z)
    
    Returns:
        model: скомпилированная модель Keras
    """
    # Входной слой - принимает изображения размером 48x48x3
    inp = layers.Input(shape=input_shape)
    x = inp
    
    # Создаем 3 сверточных блока с увеличивающимся количеством фильтров
    for i, filters in enumerate([32, 64, 128]):
        
        # Первый сверточный слой в блоке
        x = layers.Conv2D(filters, 3, padding="same")(x)
        
        x = layers.BatchNormalization()(x)
        
        x = layers.Activation("relu")(x)
        
        # Второй сверточный слой в блоке
        x = layers.Conv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        
        x = layers.MaxPooling2D()(x)
        
        # Dropout применяем только в глубоких слоях (начиная со 2-го блока
        if i >= 1:
            x = layers.Dropout(0.1)(x)
    
    # Flatten - превращаем тензор в вектор
    x = layers.Flatten()(x)
    
    # Полносвязный слой - объединяет все признаки для классификации
    x = layers.Dense(256, activation="relu")(x)
    
    # Более агрессивный Dropout перед финальным слоем
    x = layers.Dropout(0.3)(x)
    
    # Выходной слой - 26 нейронов
    out = layers.Dense(n_classes, activation="softmax")(x)
    
    model = models.Model(inp, out)
    return model

model = make_model()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 48, 48, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 48, 48, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 48, 48, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_12 (Activation)      │ (None, 48, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 48, 48, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 48, 48, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_13 (Activation)      │ (None, 48, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 24, 24, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_14 (Activation)      │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 24, 24, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 24, 24, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_15 (Activation)      │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_16 (Activation)      │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 12, 12, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_17 (Activation)      │ (None, 12, 12, 128)    │             

 Total params: 1,475,386 (5.63 MB)

 Trainable params: 1,474,490 (5.62 MB)

 Non-trainable params: 896 (3.50 KB)

In [14]:

# 4. Callbacks
cbs = [
    callbacks.EarlyStopping(patience=6, restore_best_weights=True, verbose=1),
    callbacks.ReduceLROnPlateau(patience=3, factor=0.3, min_lr=1e-5, verbose=1),
]


In [ ]:

# 5. Обучение
history = model.fit(
    train_gen,
    epochs=40,
    validation_data=val_gen,
    callbacks=cbs,
    verbose=1
)

Epoch 1/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.0947 - loss: 2.9331 - val_accuracy: 0.1300 - val_loss: 2.8135 - learning_rate: 0.0010
Epoch 2/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.1056 - loss: 2.9144 - val_accuracy: 0.1417 - val_loss: 2.7151 - learning_rate: 0.0010
Epoch 3/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.1392 - loss: 2.8177 - val_accuracy: 0.2160 - val_loss: 2.4898 - learning_rate: 0.0010
Epoch 4/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.1518 - loss: 2.7180 - val_accuracy: 0.2360 - val_loss: 2.4188 - learning_rate: 0.0010
Epoch 5/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.1696 - loss: 2.6493 - val_accuracy: 0.2750 - val_loss: 2.3303 - learning_rate: 0.0010
Epoch 6/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.2003 - loss: 2.4906 - val_accuracy: 0.3380 - val_loss: 2.1055 - learning_rate: 0.0010
Epoch 7/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.2386 - loss: 2.

In [17]:

# 6. Инференс на тесте
X_sub = np.load("images_sub.npy").astype("float32") / 255.0
preds = model.predict(X_sub, batch_size=256, verbose=1)
pred_labels = preds.argmax(axis=1).astype("int32")


196/196 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step


In [ ]:

# 7. Сохранение сабмита
sub_df = pd.DataFrame(
    {
        "Id": np.arange(len(pred_labels)),
        "Category": pred_labels,
    }
)
sub_df.to_csv("submission.csv", index=False)


Файл submission.csv сохранён.
Файл submission.csv сохранён.
